In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from tqdm import tqdm
import torch.utils.data as data

In [2]:
def calc_acc(preds,labels):
  _,preds_max = torch.max(preds,1)
  acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
  return acc

In [3]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.cnn1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=1,padding=1)
    self.cnn2= nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1) 

    self.fc1= nn.Linear(18496,128)
    self.fc2= nn.Linear(128,10)

  def forward(self, x):
    x = F.relu(self.cnn1(x))
    x = F.max_pool2d(x, kernel_size=(2,2))
    x = F.relu(self.cnn2(x))
    x = F.max_pool2d(x, kernel_size=(2,2))
    x = torch.flatten(x, start_dim=1)
    x= F.relu(self.fc1(x))
    x = torch.dropout(x,0.3,train=True)
    x = self.fc2(x)
    x = torch.softmax(x, dim=1)
    return x
  

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model().to(device)

In [5]:
batch_size=64
epochs=30
lr=0.001

In [6]:
transform = transforms.Compose([
                                # transforms.RandomRotation(10),
                                transforms.Resize((70,70)),
                                transforms.ToTensor()
                                # transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])

dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/MNIST_persian/MNIST_persian',transform=transform)
train_size=int(len(dataset)*0.8)
val_size=len(dataset)-train_size
train_data ,val_data = data.random_split(dataset,[train_size,val_size])

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [7]:
# compile
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

In [8]:
def train(model, train_data_loader,epoch):
  model.train(True)
  train_loss=0.0
  train_acc=0.0
  for images,labels in tqdm(train_data_loader):
    images=images.to(device)
    labels=labels.to(device)
    optimizer.zero_grad()
    
    preds_train = model(images)

    loss_train=loss_function(preds_train,labels) # loss_train
    loss_train.backward()

    optimizer.step()

    train_loss += loss_train
    train_acc += calc_acc(preds_train,labels)
    
  total_loss = train_loss/len(train_data_loader)
  total_acc = train_acc/len(train_data_loader)
  print(f"loss_train:{total_loss},accuracy_train:{total_acc}")


In [9]:
def test(model, val_data_loader,epoch):
  model.eval()
  test_loss=0.0
  test_acc=0.0
  for images,labels in tqdm(val_data_loader):    
    images=images.to(device)
    labels=labels.to(device)
    
    preds_test = model(images)

    loss_test=loss_function(preds_test,labels) 

    test_loss += loss_test
    test_acc += calc_acc(preds_test,labels)

  total_loss = test_loss/len(val_data_loader)
  total_acc = test_acc/len(val_data_loader)
  print(f"loss_eval:{total_loss},accuracy_eval:{total_acc}")


In [10]:
for epoch in range(epochs):
  print(f'Epoch:{epoch}')
  train(model, train_data_loader, epoch)
  test(model, val_data_loader, epoch)  

Epoch:0


100%|██████████| 15/15 [10:20<00:00, 41.34s/it]


loss_train:2.3046417236328125,accuracy_train:0.10104166666666667


100%|██████████| 4/4 [02:32<00:00, 38.01s/it]


loss_eval:2.301353693008423,accuracy_eval:0.10416666666666667
Epoch:1


...

loss_eval:1.6882514953613281,accuracy_eval:0.7981770833333334
Epoch:29


100%|██████████| 15/15 [00:10<00:00,  1.40it/s]


loss_train:1.6160534620285034,accuracy_train:0.8489583333333334


100%|██████████| 4/4 [00:01<00:00,  3.28it/s]

loss_eval:1.6980441808700562,accuracy_eval:0.7578125


In [61]:
torch.save(model.state_dict(), "/content/drive/MyDrive/MNIST_persian/model_mnist.pth")

In [58]:
import cv2
import numpy as np

model.eval()

img = cv2.imread('/content/7.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (70, 70))

from PIL import Image
PIL_image = Image.fromarray(img)

tensor = transform(PIL_image).unsqueeze(0).to(device)

# process
pred = model(tensor)

# postprocess
pred = pred.cpu().detach().numpy()
pred = np.argmax(pred)

print('prediction:',pred)

prediction: 7
